# Data Ingestion and Data Wrangling
### Introduction
This jupytehr notebook has two main tasks:
- Data Ingestion. After having found the data, this is the second step in data science, where data are in practise moved from one place to another. In this notebook means, importing the data from cvs file format into python.
- Data Wrangling. This might include several processes that are designed to transform raw data into usable data.  In this notebook data wrangling will be done by calcualting the the minimum, maximum, mean, and standard deviation of every single feature and identification of any possible outliers.

### Part 1: Data Ingestion

#### Step 1: parsing the CSV files provided by the unit
Parsing files included into the provided weather-dat.zip

#### Step 2: parsing any CSV files
Capable of parsing any other profided CSV files, considering both **input data** and **format variations**



In [1]:
import numpy as np
import pandas as pd

def data_ingestion(file):
    """Function which takes in input a string containing the input .csv file that needs to be parsed and ingested.
    It is also capable to hanldle many different malformed data:
    > missing elements in a cell
    > not omogenous data type in a specific column
    > date time format handling
    > identify columns that are numerical hence can be usedd for statistical analysis
    This function returns a tuple containing the information: 
    > if there is or not the column headers
    > which column is made of int and float elements
    > the actual data set."""    
    
    # Open and read file in input
    raw_list = []
    with open(file, mode='r', encoding='utf-8-sig') as file: 
        for row in file:
            # eliminate the space string and split based on comma
            line = row.strip().split(",")
            # append lines to create a final list
            temp = []
            temp.append(line[0])
            for val in range(1,len(line)):
                temp.append(line[val])
            raw_list.append(temp)
    
    # Using python array instead of list
    raw_array = np.array(raw_list)

    # Find out if there is or not the column headers line
    # If every element in the top row is not a digit then that row is the header columns, hence column_headers_present = 1
    n_rows = len(raw_array[:, 0])
    n_cols = len(raw_array[0, :])
    flg_digit = np.empty(shape=(n_rows, n_cols), dtype=int)
    is_header = np.empty(shape=(n_rows, 1), dtype=int)
    for row in range(n_rows):
        for col in range(n_cols):
            flg_digit[row, col] = raw_array[row, col].isdigit()
            is_header[row, 0] = not any(flg_digit[row, :]) and row == 0

    column_headers_present = 0
    # If there is column headers then split the data into header and values lists 
    if is_header[0, 0] == 1:
        column_headers_present = 1
        len_header = len(raw_array[0, :])
        header = raw_array[0, :].reshape((1, len_header))
        raw_values = raw_array[1:, :]
        n_samples = n_rows - 1 # if there is the header columns then -1 to get the samples only
    else:
        raw_values = raw_array[:, :]
        n_samples = n_rows

    # Remove quotes ("") from the values. 
    values = np.empty(shape=(n_samples, n_cols), dtype=object)
    for row in range(n_samples):
        for col in range(n_cols):
            values[row, col] = raw_values[row, col].replace('"','')
            
    # identify empty cells, then delete the entire row that contains at least one empty cell
    empty_cell = np.empty(shape=(n_samples, 1), dtype=object)
    for row in range(n_samples):
        for col in range(n_cols):
            if values[row, col] == '':
                empty_cell[row, 0] = 1
  
    raw_values = values.copy()
    values = raw_values[empty_cell[:,0] != 1]
    # recalculate n_samples since they might have been reduced
    n_samples = len(values[:, 0])
    if n_samples == 0:
        print("Sorry, I can't progress it, there are zero valid samples available!")
        return

    # Detect if a column contains date and time and which one is.
    # Convert date and time into standard format.
    raw_values = values.copy()
    values = np.empty(shape=(n_samples, n_cols), dtype=object)
    time_date_flg = np.empty(shape=(n_samples, n_cols), dtype=int)
    for col in range(n_cols):
        for row in range(n_samples):
            try:  
                # Convert date and time into standard date time format
                values[row, col] = pd.to_datetime(raw_values[row, col])
                # Set time_date_flg to 1 which means that the row contains date time field
                time_date_flg[row, col] = 1
            except:
                # Set time_date_flg to 0 which means that the row does not contain date time field
                time_date_flg[row, col] = 0
                try:
                    # If cannot convert date time it implies the column does not contain date time field
                    values[row, col] = float(raw_values[row, col])
                except:
                    values[row, col] = raw_values[row, col]
                    if column_headers_present == 1:
                        row_new = row + 1
                        col_new = col + 1
                        print(f"Could not convert row = {row_new} col ={col_new} to float")
                    else:
                        row_new = row
                        col_new = col
                        print(f"Could not convert row = {row_new} col ={col_new} to float")                        
                    
    # find for every cell if it is numeric (integer or float) or not
    is_int_float = np.empty(shape=(n_samples, n_cols), dtype=int)
    for row in range(n_samples):
        for col in range(n_cols):
            if ( type(values[row, col]) == int ) or ( type(values[row, col]) == float ):
                is_int_float[row, col] = 1
            else:
                is_int_float[row, col] = 0   

    # find for every feature (column) if it is more frequent the numeric value (integer or float) or not
    count_int_float = np.empty(shape=(1, n_cols), dtype=int)
    common_int_float = np.empty(shape=(1, n_cols), dtype=int)
    for col in range(n_cols):
        # if = 1 it means that the most frequent type of value in the column is integer or floating
        common_int_float[0, col] = np.argmax(np.bincount(is_int_float[:, col]))

    # take out the rows that contains values with type different from the most common type in that column
    wrong_type_cell = np.empty(shape=(n_samples, 1), dtype=object)
    raw_values = values.copy()
    for col in range(n_cols):
        for row in range(n_samples):
                # assign value 1 to the rows that have at least one cell which is not alligned with the rest of cells in their belong columns
            if (common_int_float[0, col] == 1) and (type(raw_values[row, col]) != float) and (type(raw_values[row, col]) != int): 
                wrong_type_cell[row, 0] = 1  
                
    # keep only the rows that don't have a cell which is not alligned with the most common type of it belonging column
    values = raw_values[wrong_type_cell[:,0] != 1]
    # recalculating the number of sample in case one or more rows have been deleted
    n_samples = len(values[:, 0])
    if n_samples == 0:
        print("Sorry, I can't progress it, there are zero valid samples available!")
        return
    
    counts_time_date_flg = np.empty(shape=(1, n_cols), dtype=int)
    for col in range(n_cols):
        counts_time_date_flg[0, col] = np.count_nonzero(time_date_flg[:, col] == 1)        

    raw_values = values.copy()
    for col in range(n_cols):
        if counts_time_date_flg[0, col] < n_samples and counts_time_date_flg[0, col] > 0:
            print(f"Column '{header[0, col]}' is an incomplete date time column")
            
    # malformed data. Header column should have the same number of fieds as the values
    # malformed data. Every sample should have the same numer of fields
    len_row_val_calc = np.empty(shape=(n_samples, 1), dtype=int)
    for row in range(n_samples):
        len_row_val_calc[row, 0] = len(values[row, :])

    # length of every single row of the dataset
    len_row_val = np.concatenate(len_row_val_calc)

    # It gives back the most frequent number of fields in the data (length of the rows)
    count_fields = np.bincount(len_row_val)  
    common_fields_n = np.argmax(count_fields)

    # Check if the header columns has the right number of fields compare to the rest of the data
    if column_headers_present == 1 and len_header != common_fields_n:
        print("The columns header has different number of field from the values!")
    # Check if every single sample has the right number of fields compare to the rest of the values
    for row in range(n_samples):
        if len_row_val_calc[row, 0] != common_fields_n:
            print(f"Row {row} has different number of fields respect to the rest of values, please correct it!")
    
    # if there is a column header then concatenate it to the rest of the values
    if column_headers_present == 1:
        data = np.concatenate((header, values), axis=0)
    else:
        data = values
    return column_headers_present, common_int_float, data 

For the **USER**, please define the name of the csv file that you want to process 'file_name':

In [2]:
# Input the file name to be processed
file_name = "barometer-1617.csv" 

data_ingest = data_ingestion(file_name)

### Part 2: Data Wrangling

#### Step 1: minimum, maximum, mean, and standard deviation
Computing minimum, maximum, mean, and standard deviation for file weather-dat.zip

#### Step 2: with outliers: minimum, maximum, mean, and standard deviation
Computing minimum, maximum, mean, and standard deviation for a new file that contains outliers

In [3]:
def data_wrangling(input_data):
    """Function which takes in input a tuple containing: the information if there is or not the column headers and the data set. 
    It returns a dataframe containing the minimum, maximum, mean and standard deviation of every single column (feature)."""
    
    column_headers_present = input_data[0]
    common_int_float = input_data[1]
    data = input_data[2]
    #print("data", data)

    n_rows = len(data[:, 0])
    n_cols = len(data[0, :])
    if column_headers_present == 1:
        raw_header = data[0, :]
        raw_values = data[1:, :]
        n_samples = n_rows - 1
    else:
        raw_values = data[:, :]
        n_samples = n_rows
    
    # keep only the header columns and values that are integer and float
    header = raw_header[common_int_float[0, :] == 1]
    n_cols_stats = len(header[:])
    values = np.empty(shape=(n_samples, n_cols_stats), dtype=object)
    for row in range(n_samples):
        values[row, :] = raw_values[row, :][common_int_float[0, :] == 1]
    
    # Calculate the basic statistics: minimum, maximum, mean and standard deviation for every feature (column)
    min_val = np.empty(shape=(1, n_cols_stats), dtype=object)
    max_val = np.empty(shape=(1, n_cols_stats), dtype=object)
    mean_val = np.empty(shape=(1, n_cols_stats), dtype=object)
    std_val = np.empty(shape=(1, n_cols_stats), dtype=object)
    for col in range(n_cols_stats):
        min_val[0, col] = np.min(values[:, col])
        max_val[0, col] = np.max(values[:, col])
        mean_val[0, col] = np.mean(values[:, col])
        std_val[0, col] = np.std(values[:, col])
    
    statistics_names = ["min", "max", "mean", "stand. dev."]
    statistics_data = np.concatenate((min_val, max_val, mean_val, std_val), axis=0)

    # Create a data frame to better handle the col and row labels for showing the basic statistics
    data_frame_statist = pd.DataFrame(statistics_data, statistics_names, header)
    
    return data_frame_statist

In [4]:
# Using the ingested data from the previous phase to apply data wrangling
print("file name:", file_name)
print()
if data_ingest is None:
    print ("No statistics available because 'data_ingest' is empty")

else:
    data_wrang = data_wrangling(data_ingest)
    print(data_wrang)

file name: barometer-1617.csv

                  "Baro"
min                979.6
max               1035.6
mean         1009.998873
stand. dev.     9.855751
